In [1]:
class HarvestEnv(gym.Env):
    def __init__(self, n_crops=2, step_per_year=12):
        # state dims, budgets, parameters
        self.n_crops = n_crops
        self.step_per_year = step_per_year
        self.reset()

    def reset(self):
        # initialize soil, crops, cash, market price (randomized)
        self.t = 0
        self.soil_health = 0.8
        self.inventory = np.zeros(self.n_crops)
        self.cash = 1000.0
        self.market_price = init_prices()
        self.crop_states = init_crops()
        return self._get_obs()

    def step(self, action):
        # action: vector [water_allocs..., fert_allocs..., labor_allocs..., invest, sell_fracs...]
        action = self._project_action(action)  # ensure feasibility
        costs = compute_costs(action)
        self.cash -= costs
        # crop growth
        self.crop_states = update_crops(self.crop_states, action, self.weather(), self.soil_health)
        harvest = compute_harvest(self.crop_states)   # instant harvest added to inventory
        self.inventory += harvest
        # apply selling
        sold = self.inventory * action.sell_fracs
        revenue = (sold * self.market_price).sum()
        self.inventory -= sold
        self.cash += revenue
        # update soil health
        self.soil_health = update_soil(self.soil_health, harvest, action.invest)
        # update market price
        self.market_price = update_price(self.market_price)
        reward = revenue - costs - storage_cost(self.inventory)
        self.t += 1
        done = (self.t >= self.step_per_year * num_years)
        return self._get_obs(), reward, done, info


NameError: name 'gym' is not defined